In [1]:
import plotly.graph_objects as go
import numpy as np

"""
Created on Wed Mar  6 16:11:37 2024

@author: Gino Del Ferraro

"""
import sys
import os 

dir_current = %pwd
dir_utils = os.path.abspath(os.path.join(dir_current, '..', 'utils-tools'))

# print(dir_utils)
sys.path.append(dir_utils)


import numpy as np
import pickle
from utilities_ketamine_analysis_v8 import *
from utils_signal_processing import *
from utils_plotting import *
from utils_general import *
from utils_phase_frequency import *
import matplotlib.gridspec as gridspec
import time 
from mpl_toolkits.mplot3d import Axes3D
from matplotlib.animation import FuncAnimation

binFullPath = r'C:\Users\fentonlab\Desktop\Gino\LFPs'
main_dir = r'C:\Users\fentonlab\Desktop\Gino\LFPs\HPC'
HPC_path_file = os.path.join(r'C:\Users\fentonlab\Desktop\Gino\LFPs','HPC_lfp_paths.file')
PFC_path_file = os.path.join(r'C:\Users\fentonlab\Desktop\Gino\LFPs','PFC_lfp_paths.file')

sess = 2

# ====== Load rec path 
rec = load_rec_path(binFullPath,HPC_path_file,PFC_path_file,"HPC",sess)
idx_cell_HPC = load_list_channel_CA1(rec, sess, "HPC")
freq = np.logspace(0.32, 2.48, num=70, endpoint=True)

# frequency range/bands
theta = np.where((freq>=2) & (freq <=15)) # [2, 15] Hz
low_gamma = np.where((freq>=30) & (freq <=60)) # [30, 60] Hz
high_gamma = np.where((freq>60) & (freq <=100)) # # (60, 100] Hz

"""
Compute Mean Abs Error for freq-phase map wrt to Baseline-Low dose, Baseline-Mid dose, Baseline-High dose
in the theta, low gamma, and high gamma frequency range 
"""
mae_sess = []
for idx in range(0,len(idx_cell_HPC)):
    cell = idx_cell_HPC[idx]
    pdf_sm = load_matrices_freq_phase_4_by_4(rec, sess, "HPC", cell, idx) 
    mae_sess.append( mean_abs_error_freq_phase_maps(pdf_sm,theta,low_gamma,high_gamma) )
    

# vector = create_array_for_3D_plots(mae_sess,idx_cell_HPC,0,'theta')
# plot_3D_mean_abs_error_all_cells(vector)

vector_BL = create_array_for_3D_plots_axis_freq_band(mae_sess,idx_cell_HPC,0,'BL')
vector_BM = create_array_for_3D_plots_axis_freq_band(mae_sess,idx_cell_HPC,0,'BM')
vector_BH = create_array_for_3D_plots_axis_freq_band(mae_sess,idx_cell_HPC,0,'BH')

Loading file in:  Z:\NeuroPix\Ketamine\2022-08-01_04-30-00_M015_RSK_mPFC_HPC_3_10_30mpk\2022-08-01_04-30-00_M015_RSK_mPFC_HPC_3_10_30mpk_g0_imec1\2022-08-01_04-30-00_M015_RSK_mPFC_HPC_3_10_30mpk_g0_t0.imec1.lf.bin


In [2]:
remove_idx = [5,6,8,9,10,11,12,17,19,20,21,22,24,26,28,29,30,31,32,37,38,39,44,45]

In [3]:
# Creating the 3D scatter plot -- working only on Jupyter notebook
def animated_3D_scatter_plot(vector,title_name,remove_idx, remove=False):

    X = vector[:,1]*100
    Y = vector[:,2]*100
    Z = vector[:,3]*100
    
    labels = [str(i) for i in range(0,len(X))]  # List of strings '1', '2', ...
    
    
    if remove: # remove cells with 'strange behavior'
        X = np.delete(X, remove_idx)
        Y = np.delete(Y, remove_idx)
        Z = np.delete(Z, remove_idx)
        labels = np.delete(labels, remove_idx)
    
    fig = go.Figure(data=[go.Scatter3d(
        x=X,
        y=Y,
        z=Z,
        mode='markers+text',
        marker=dict(
            size=8,
            color=np.linspace(0, 1, vector.shape[0]),
            colorscale='Viridis', # Choose a colorscale
            opacity=0.8
        ),
        text = labels,
        hoverinfo='text+name'
    )])

    # Set plot layout
    fig.update_layout(title=title_name,
                      scene=dict(
                          xaxis_title='Theta',
                          yaxis_title='Low Gamma',
                          zaxis_title='High Gamma',
                          camera=dict(
                              eye=dict(x=1.25, y=1.25, z=1.25)  # Adjust camera view
                          )
                      ),
                      margin=dict(l=0, r=0, b=0, t=30),
                      # Specify the width and height of the figure (in pixels)
                      width=500,  # Customize the width
                      height=500  # Customize the height
                     )
    fig.show()

In [9]:
remove_idx = [5,6,8,9,10,11,12,17,19,20,21,22,24,26,28,29,30,31,32,37,38,39,44,45]

In [15]:
animated_3D_scatter_plot(vector_BL,'base-LOW',remove_idx,True)

In [16]:
animated_3D_scatter_plot(vector_BM,'base-MID',remove_idx,True)

In [18]:
animated_3D_scatter_plot(vector_BH,'base-HIGH',remove_idx,False)